In [36]:
import configparser
import datetime as dt
import sys
import collections
from pathlib import Path
from typing import Union, Optional, List, Tuple

import codebook.EDA as EDA
import codebook.clean as clean
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy
from sqlalchemy import func, distinct

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
plt.style.use('raph-base')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 30)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

np.random.seed(666)

In [3]:
sys.path.append(str(Path.cwd().parent))

from src.db_declaration import Base, Artist, CreditTrx, Genre, Label, Record
from src import db_functions
from src import utils

In [4]:
print(sys.executable)
print(sys.version)
print(f'sqlalchemy {sqlalchemy.__version__}')

C:\Users\r2d4\miniconda3\envs\py3\python.exe
3.8.3 (default, May 19 2020, 06:50:17) [MSC v.1916 64 bit (AMD64)]
sqlalchemy 1.3.17


## Connect To And (Re-)Create DB

In [5]:
CONFIG_PATH = Path.cwd().parent / "config.cfg"

In [6]:
path_to_db = utils.read_config_return_str(CONFIG_PATH, "SQLITE_DEV")
engine = utils.create_engine(path_to_db)
session = utils.create_session(engine)
utils.create_DB_anew(engine, Base)

C:\Users\r2d4\OneDrive\code\projects\20-02_disco\dev\DeafDiscoBase.db


In [7]:
def load_some_albums_from_xlsx(
    filepath: Union[Path, str],
    n: int=3,
    random_state: int=5,
) -> pd.DataFrame:
    """Load the original album collection file into a dataframe.
    You can specify a list of genres you want to include
    (defaults to None).
    """
    df = pd.read_excel(filepath, engine="openpyxl")
    return df.sample(n=n, random_state=random_state)


def clean_collection(collection_df):
    collection_df = clean.prettify_column_names(collection_df)
    collection_df.dropna(thresh=8, inplace=True)
    collection_df.rename(columns={"format": "record_format"}, inplace=True)
    collection_df["label"].fillna("NA", inplace=True)  # because it creates a foreign key
    collection_df["vinyl_color"] = np.nan  # just for aesthetics ...
    collection_df["artist_country"] = None
    collection_df["credit_value"] = 0
    collection_df["trx_type"] = "Initial Load"
    return collection_df

In [8]:
path_to_collection = utils.read_config_return_str(CONFIG_PATH, "COLLECTION")
collection_df = load_some_albums_from_xlsx(path_to_collection)
collection_df = clean_collection(collection_df)

In [9]:
collection_df

,active,artist,title,record_format,year,genre,price,purchase_date,vinyl_color,lim_edition,number,label,digitized,remarks,rating,artist_country,credit_value,trx_type
28,1.00,Emperor,s/t,MLP,"1,993.00",Black Metal,190.00,2014-08-01,nan,NaN,NaN,NA,False,NaN,nan,None,0,Initial Load
244,1.00,Gatecreeper,Deserted,LP,"2,019.00",Death Metal,70.00,2020-01-01,nan,100,NaN,Relapse Records,True,NaN,8.00,None,0,Initial Load
123,1.00,Agnosy,Traits of The Past,LP,"2,014.00",Crust,15.00,2017-08-01,nan,NaN,NaN,NA,True,NaN,nan,None,0,Initial Load


In [10]:
# Create an additional test_record

In [11]:
test_record = load_some_albums_from_xlsx(path_to_collection, 1, 1)
test_record = clean_collection(test_record)
test_record = test_record.to_dict(orient="records")[0]
test_record["trx_type"] = "Purchase"
test_record["credit_value"] = 1
test_record

{'active': 1.0,
 'artist': 'Witch Vomit',
 'title': 'Poisoned Blood',
 'record_format': 'MLP',
 'year': 2017.0,
 'genre': 'Death Metal',
 'price': 15.0,
 'purchase_date': Timestamp('2020-06-01 00:00:00'),
 'vinyl_color': nan,
 'lim_edition': nan,
 'number': nan,
 'label': '20 Buck Spin',
 'digitized': True,
 'remarks': nan,
 'rating': 8.0,
 'artist_country': None,
 'credit_value': 1,
 'trx_type': 'Purchase'}

### Initial Load of 3 Records

In [12]:
def insert_df_with_sqlalchemy_orm(session, df):
    for x in df.to_dict("records"):
        db_functions.add_new_record(session, x)

In [13]:
insert_df_with_sqlalchemy_orm(session, collection_df)

assert session.query(Record).count() == 3

C:\Users\r2d4\miniconda3\envs\py3\lib\site-packages\sqlalchemy\sql\sqltypes.py:660: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  util.warn(


### Insertion of 2 Credit Addition Trx

In [14]:
# to_delete = session.query(CreditTrx).filter(CreditTrx.credit_trx_id == 8).one()
# session.delete(to_delete)
# session.commit()

NoResultFound: No row was found for one()

In [15]:
# Initial trx, 11 days ago

addition_trx = CreditTrx(
    credit_trx_date=dt.datetime.today().date() - dt.timedelta(11),
    credit_trx_type="Addition",
    credit_value=1,
    credit_saldo=1,
    record_id=np.nan
)
session.add(addition_trx)

# And a regular interval addition
db_functions.add_regular_credits(session)

session.commit()

Creating 'Addition' Trx for: 2021-01-08


In [16]:
session.query(CreditTrx).all()

[<CreditTrx(credit_trx_id=1, credit_trx_date=2014-08-01, credit_trx_type=Initial Load, credit_value=0.0, credit_saldo=0.0, record_id=1)>,
 <CreditTrx(credit_trx_id=2, credit_trx_date=2020-01-01, credit_trx_type=Initial Load, credit_value=0.0, credit_saldo=0.0, record_id=2)>,
 <CreditTrx(credit_trx_id=3, credit_trx_date=2017-08-01, credit_trx_type=Initial Load, credit_value=0.0, credit_saldo=0.0, record_id=3)>,
 <CreditTrx(credit_trx_id=4, credit_trx_date=2020-12-29, credit_trx_type=Addition, credit_value=1.0, credit_saldo=1.0, record_id=None)>,
 <CreditTrx(credit_trx_id=5, credit_trx_date=2021-01-08, credit_trx_type=Addition, credit_value=1.0, credit_saldo=2.0, record_id=None)>]

### Insertion of new Record

In [18]:
db_functions.add_new_record(session, test_record)

assert session.query(Record).count() == 4

In [19]:
session.query(CreditTrx).all()[-2:]

[<CreditTrx(credit_trx_id=5, credit_trx_date=2021-01-08, credit_trx_type=Addition, credit_value=1.0, credit_saldo=2.0, record_id=None)>,
 <CreditTrx(credit_trx_id=6, credit_trx_date=2020-06-01, credit_trx_type=Purchase, credit_value=-1.0, credit_saldo=1.0, record_id=4)>]

### Removal of Existing Record

Necessary cols: trx_type, credit_value, title, artist, year, date

In [27]:
test_removal = {
    "trx_type": "Removal",
    "credit_value": 1,
    "artist": "Emperor",
    "title": "s/t",
    "year": 1993,
    "date": dt.datetime.today().date()
}

In [30]:
db_functions.set_record_to_inactive(session, test_removal)

Status of record 's/t' by Emperor is already 0, please check.


In [42]:
session.query(func.count(distinct(Record.active))).all()
session.query(func.count(Record.record_id)).group_by(Record.active).all()

[(2)]

[(1), (3)]

In [43]:
session.query(CreditTrx).all()[-2:]

[<CreditTrx(credit_trx_id=6, credit_trx_date=2020-06-01, credit_trx_type=Purchase, credit_value=-1.0, credit_saldo=1.0, record_id=4)>,
 <CreditTrx(credit_trx_id=7, credit_trx_date=2021-01-09, credit_trx_type=Removal, credit_value=1.0, credit_saldo=2.0, record_id=1)>]

### Reactivation of inactive Record

In [ ]:
ATTENTION It has to be possible ro re-add inactive records! (and to pay for it in credits!)

## Query DB

In [ ]:
for result in session.query(Record).filter(Record.title == "Death Lust").all():
    result

In [ ]:
session.query(Artist).count()
session.query(Artist).all()[:5]

In [ ]:
session.query(Genre).count()
session.query(Genre).all()

In [ ]:
result = session.query(CreditTrx.credit_saldo).all()

np.array(result).min()
np.array(result).max()

In [ ]:
from src.db_declaration import GenreLabelLink, ArtistLabelLink
from pprint import pprint

session.query(func.count(ArtistLabelLink.artist_id)).all()[0][0]
session.query(ArtistLabelLink).count()

for result in session.query(Artist).all()[:5]:
    pprint(result)

In [ ]:
session.query(func.sum(CreditTrx.credit_value)).all()[0][0]

## Dev Credit Addition

In [ ]:
for id_ in session.query(CreditTrx.credit_trx_id).all():
#     print(id_[0])
    to_delete = session.query(CreditTrx).filter(CreditTrx.credit_trx_id == id_[0]).one()
    session.delete(to_delete)

In [ ]:
# Add fake Addition trx

fake_trx = CreditTrx(
    credit_trx_date=dt.datetime(year=2020, month=11, day=2),
    credit_trx_type="Addition",
    credit_value=1,
    credit_saldo=np.array(session.query(CreditTrx.credit_value).all()).sum() + 1,
    record_id=np.nan
)
session.add(fake_trx)

fake_trx = CreditTrx(
    credit_trx_date=dt.datetime(year=2020, month=11, day=1),
    credit_trx_type="Addition",
    credit_value=1,
    credit_saldo=np.array(session.query(CreditTrx.credit_value).all()).sum() + 1,
    record_id=np.nan
)
session.add(fake_trx)

fake_trx = CreditTrx(
    credit_trx_date=dt.datetime(year=2020, month=12, day=1),
    credit_trx_type="Test",
    credit_value=1,
    credit_saldo=np.array(session.query(CreditTrx.credit_value).all()).sum() + 1,
    record_id=np.nan
)
session.add(fake_trx)
# session.commit()

In [ ]:
# Check
# session.commit()
session.query(CreditTrx).all()

In [ ]:
last_addition_date = (session
    .query(CreditTrx.credit_trx_date)
    .filter(CreditTrx.credit_trx_type == "Addition")
    .order_by(CreditTrx.credit_trx_date.desc())
    .first()
)[0]

days_since_last = (dt.date.today() - last_addition_date).days

In [ ]:
last_addition_date
days_since_last

In [ ]:
# while days_since_last >= 10:
#     fake_trx = CreditTrx(
#         credit_trx_date=last_addition_date + + dt.timedelta(days=10),
#         credit_trx_type="Addition",
#         credit_value=1,
#         credit_saldo=np.array(session.query(CreditTrx.credit_value).all()).sum() + 1,
#         record_id=np.nan
#     )
    
#     session.add(fake_trx)
    
#     last_addition_date = (session
#         .query(CreditTrx.credit_trx_date)
#         .filter(CreditTrx.credit_trx_type == "Addition")
#         .order_by(CreditTrx.credit_trx_date.desc())
#         .first()
#     )[0]
    
#     days_since_last = (dt.date.today() - last_addition_date).days

# session.commit()

In [ ]:
def _get_days_since_last_addition() -> Tuple[dt.date, int]:
    """Return the date of and the number of days since the 
    last transaction with type 'Addition' stored in the 
    CreditTrx table. (This is called within 'add_credit').
    """
    last_addition_date = (session
        .query(CreditTrx.credit_trx_date)
        .filter(CreditTrx.credit_trx_type == "Addition")
        .order_by(CreditTrx.credit_trx_date.desc())
        .first()
    )[0]

    days_since_last = (dt.date.today() - last_addition_date).days
    
    return last_addition_date, days_since_last


def add_regular_credits(interval_days: int =10):
    """Every x days a new credit is added (to be spent
    on purchasing new records). This function checks
    the delta in days since the last addition and inserts
    the necessary credit transactions depending on the
    defined interval.
    """
    last_addition_date, days_since_last = _get_days_since_last_addition()
    
    while days_since_last >= 10:
        print(last_addition_date)
        addition_trx = CreditTrx(
            credit_trx_date=last_addition_date + dt.timedelta(days=interval_days),
            credit_trx_type="Addition",
            credit_value=1,
            credit_saldo=np.array(session.query(CreditTrx.credit_value).all()).sum() + 1,
            record_id=np.nan
        )
        session.add(addition_trx)
        last_addition_date, days_since_last = _get_days_since_last_addition()
    
    session.commit()

In [ ]:
add_regular_credits()

In [ ]:
session.query(CreditTrx).all()

In [ ]:
# session.rollback()

In [ ]:
session.close()